<a href="https://colab.research.google.com/github/ssunggun2/2022_dacon_NLI/blob/main/Private_6th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google import colab
colab.drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1. 모듈 설치 및 import

In [2]:
!pip install transformers datasets
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install googletrans==4.0.0-rc1
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 3.8 MB 8.0 MB/s 
     |████████████████████████████████| 311 kB 53.4 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 55.0 MB/s 
     |████████████████████████████████| 6.5 MB 46.8 MB/s 
     |████████████████████████████████| 596 kB 75.7 MB/s 
     |████████████████████████████████| 212 kB 53.7 MB/s 
     |████████████████████████████████| 134 kB 47.2 MB/s 
     |████████████████████████████████| 1.1 MB 58.4 MB/s 
     |████████████████████████████████| 144 kB 56.7 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 271 kB 97.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 963 kB 8.2 MB/s 
     |████████████████████████████████| 138 kB 75.2 MB/s 
     |████████████████████████████████| 359 k

In [3]:
import os
import time
import random
import googletrans
from googletrans import Translator

import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, TFAutoModel, AdamWeightDecay,  AutoModelForSequenceClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.callbacks import EarlyStopping

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


seed

In [4]:
def seed_everything(seed: int = 2022):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything()


2. DATA LOAD

2-1 기존 데이터, dev 추가 데이터 : train_total for model1

In [5]:
from zipfile import ZipFile

In [6]:
zip_dir = '/content/drive/MyDrive/화요일 스터디 폴더/dacon'
zip_name = 'open.zip'
zip_fPath = os.path.join(zip_dir, zip_name)

with ZipFile(zip_fPath, 'r') as zip1:
    zip1.printdir()
    zip1.extractall() 

File Name                                             Modified             Size
open/sample_submission.csv                     2022-01-25 15:19:44        18894
open/test_data.csv                             2022-01-25 15:19:46       320363
open/train_data.csv                            2022-01-25 17:30:34      4855176


In [7]:
train_df = pd.read_csv('/content/open/train_data.csv')
test_df = pd.read_csv('/content/open/test_data.csv')
submission_df = pd.read_csv('/content/open/sample_submission.csv')
train_df.head()

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral


추가 데이터는 airbnb 데이터, wikinews

In [8]:
data_path = zip_dir + "/data/translate/" #번역본의 위치
train_path2 = zip_dir + "/klue-nli-v1.1/klue-nli-v1.1_dev.json"

In [9]:
train_path2

'/content/drive/MyDrive/화요일 스터디 폴더/dacon/klue-nli-v1.1/klue-nli-v1.1_dev.json'

In [10]:
train_additional = pd.read_json(train_path2)
train_additional.head()

,guid,source,premise,hypothesis,gold_label,author,label2,label3,label4,label5
0,klue-nli-v1_dev_00000,airbnb,흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.,어떤 방에서도 흡연은 금지됩니다.,contradiction,contradiction,contradiction,contradiction,contradiction,contradiction
1,klue-nli-v1_dev_00001,airbnb,10명이 함께 사용하기 불편함없이 만족했다.,10명이 함께 사용하기 불편함이 많았다.,contradiction,contradiction,contradiction,contradiction,contradiction,contradiction
2,klue-nli-v1_dev_00002,airbnb,10명이 함께 사용하기 불편함없이 만족했다.,성인 10명이 함께 사용하기 불편함없이 없었다.,neutral,neutral,neutral,neutral,entailment,neutral
3,klue-nli-v1_dev_00003,airbnb,10명이 함께 사용하기 불편함없이 만족했다.,10명이 함께 사용하기에 만족스러웠다.,entailment,entailment,entailment,entailment,entailment,entailment
4,klue-nli-v1_dev_00004,airbnb,10층에 건물사람들만 이용하는 수영장과 썬베드들이 있구요.,건물사람들은 수영장과 썬베드를 이용할 수 있습니다.,entailment,entailment,entailment,entailment,entailment,entailment


In [11]:
train_additional['index'] = train_additional.index
train_additional = train_additional[['index','premise','hypothesis','gold_label']]
train_additional.columns = ['index','premise','hypothesis','label'] # <= concate 위해서

In [12]:
train_total = pd.concat([train_df,train_additional])
train_total.drop('index', axis = 1,inplace = True)

2-2 번역 데이터 준비

Premise KOR > ENG (using PAPAGO, Googletrans)
papago 크롤링은 뉴스 토픽 분류 AI 경진대회 최종 3th Kerry님의 코드를 참고했습니다 🙇

In [13]:
import time
from selenium import webdriver
# 코랩에서 크롬 웹드라이버 돌릴 수 있게
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  return driver

driver = chrome_setting()

In [14]:
# train_df['premise'] = train_df['premise'].str.replace('%', '퍼센트')
# train_df['hypothesis'] = train_df['hypothesis'].str.replace('&', ' 그리고 ')
# train_additional['premise'] = train_additional['premise'].str.replace('%', '퍼센트')
# train_additional['hypothesis'] = train_additional['hypothesis'].str.replace('&', ' 그리고 ')
# #파파고 크롤링
# def kor_to_trans(text_data, trans_lang,start_index,final_index):

#   target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

#   for i in tqdm(range(start_index,final_index)): 
    
#     if (i!=0)&(i%99==0):
#       driver.implicitly_wait(2)
#       print('{}th : '.format(i), backtrans)
#       np.save(data_path+'p_kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
#     try:
#       driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
#       time.sleep(1.5)
#       element=WebDriverWait(driver, 10).until(target_present)
#       time.sleep(0.1)
#       backtrans = element.text 

#       if (backtrans=='')|(backtrans==' '):
#         element=WebDriverWait(driver, 10).until(target_present)
#         backtrans = element.text 
#         trans_list.append(backtrans)
#       else:
#         trans_list.append(backtrans)
    
#     except:
#       trans_list.append('')
# trans_list=[]
# kor_to_trans(train_df['premise'], 'en',0,2500)
# np.save(data_path+'p_kor_to_eng_train_0_2500.npy',trans_list)

# trans_list=[]
# kor_to_trans(train_df['premise'], 'en',2500,5000)
# np.save(data_path+'p_kor_to_eng_train_2500_5000.npy',trans_list)

# trans_list=[]
# kor_to_trans(train_df['premise'], 'en',5000,7500)
# np.save(data_path+'p_kor_to_eng_train_5000_7500.npy',trans_list)

# trans_list=[]
# kor_to_trans(train_df['premise'], 'en',7500,10000)
# np.save(data_path+'p_kor_to_eng_train_7500_10000.npy',trans_list)

# trans_list=[]
# kor_to_trans(train_df['premise'], 'en',10000,12500)
# np.save(data_path+'p_kor_to_eng_train_10000_12500.npy',trans_list)

# trans_list=[]
# kor_to_trans(train_df['premise'], 'en',12500,15000)
# np.save(data_path+'p_kor_to_eng_train_12500_15000.npy',trans_list)

# trans_list=[]
# kor_to_trans(train_df['premise'], 'en',15000,17500)
# np.save(data_path+'p_kor_to_eng_train_15000_17500.npy',trans_list)

# trans_list=[]
# kor_to_trans(train_df['premise'], 'en',17500,20000)
# np.save(data_path+'p_kor_to_eng_train_17500_20000.npy',trans_list)

# trans_list=[]
# kor_to_trans(train_df['premise'], 'en',20000,22500)
# np.save(data_path+'p_kor_to_eng_train_20000_22500.npy',trans_list)

# trans_list=[]
# kor_to_trans(train_df['premise'], 'en',22500,len(train_df))
# np.save(data_path+'p_kor_to_eng_train_22500_all.npy',trans_list) #24998개 번역
# eng_data0 = np.load(data_path+'p_kor_to_eng_train_0_2500.npy')
# eng_data1 = np.load(data_path+'p_kor_to_eng_train_2500_5000.npy')
# eng_data2 = np.load(data_path+'p_kor_to_eng_train_5000_7500.npy')
# eng_data3 = np.load(data_path+'p_kor_to_eng_train_7500_10000.npy')
# eng_data4 = np.load(data_path+'p_kor_to_eng_train_10000_12500.npy')
# eng_data5 = np.load(data_path+'p_kor_to_eng_train_12500_15000.npy')
# eng_data6 = np.load(data_path+'p_kor_to_eng_train_15000_17500.npy')
# eng_data7 = np.load(data_path+'p_kor_to_eng_train_17500_20000.npy')
# eng_data8 = np.load(data_path+'p_kor_to_eng_train_20000_22500.npy')
# eng_data9 = np.load(data_path+'p_kor_to_eng_train_22500_all.npy')

# eng_data=[*eng_data0,*eng_data1,*eng_data2,*eng_data3,*eng_data4,*eng_data5,*eng_data6,*eng_data7,*eng_data8,*eng_data9]
# eng_data=pd.DataFrame(eng_data,columns=['eng_premise'])
# back_train=pd.concat([train_df,eng_data],axis=1)
# no_back_train = back_train[(back_train['eng_premise']=='')]
# #Googletrans 

# def translate_google_en(df, col, lang='en'):
 
#   translator = Translator()
#   df['eng'] = [translator.translate(i, src='ko', dest=lang).text for i in df[col]]   # 한영 변환을 새로운 칼럼 'eng'에 담습니다
   
#   tmp1 = df.drop(['eng'], axis=1)                                                    # 2개의 temporary 데이터프레임을 생성해 원본과 한영번역 데이터프레임을 합칩니다
#   tmp2 = df.drop([col], axis=1)
#   tmp2.rename(columns={'eng':'data'}, inplace=True)
#   result = pd.concat([tmp1,tmp2], ignore_index=True)
#   result = result.drop_duplicates()
#   return result    
# back_train2 = translate_google_en(no_back_train, 'premise')
# back_train2 = back_train2[['index', 'premise', 'hypothesis', 'label', 'data' ]][len(no_back_train):]
# back_train2 = back_train2.reset_index(drop=True)
# # 인덱스 순서대로 삽입
# for i,j in zip(back_train2['index'], range(len(no_back_train))) :
#   back_train['eng_premise'].iloc[i] = back_train2['data'].iloc[j]
# back_train.to_csv(data_path + "premise_translate_eng.csv", index=False) 
# Premise ENG > KOR
# # 영한 파파고 번역기 크롤링
# def trans_to_kor(transed_list, transed_lang,start_index,final_index): 
  
#   target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

#   for i in tqdm(range(start_index,final_index)): 
    
#     if (i!=0)&(i%99==0):
#       time.sleep(1.5)
#       print('{}th : '.format(i), backtrans)
#       np.save(data_path+'premise_eng_to_kor_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
#     try:
#       driver.get('https://papago.naver.com/?sk=en&tk='+transed_lang+'&st='+transed_list[i])
#       time.sleep(2)
#       element=WebDriverWait(driver, 10).until(target_present)
#       time.sleep(0.2)
#       backtrans = element.text 

#       if (backtrans=='')|(backtrans==' '):
#         element=WebDriverWait(driver, 10).until(target_present)
#         backtrans = element.text 
#         trans_list.append(backtrans)
#       else:
#         trans_list.append(backtrans)
    
#     except:
#       trans_list.append('')
# trans_list=[]
# trans_to_kor(back_train['eng_premise'], 'ko',0,2500)
# np.save(data_path+'premise_eng_to_kor_train_0_2500.npy',trans_list)

# trans_list=[]
# trans_to_kor(back_train['eng_premise'], 'ko',2500,5000)
# np.save(data_path+'premise_eng_to_kor_train_2500_5000.npy',trans_list)

# trans_list=[]
# trans_to_kor(back_train['eng_premise'], 'ko',5000,7500)
# np.save(data_path+'premise_eng_to_kor_train_5000_7500.npy',trans_list)

# trans_list=[]
# trans_to_kor(back_train['eng_premise'], 'ko',7500,10000)
# np.save(data_path+'premise_eng_to_kor_train_7500_10000.npy',trans_list)

# trans_list=[]
# trans_to_kor(back_train['eng_premise'], 'ko',10000,12500)
# np.save(data_path+'premise_eng_to_kor_train_10000_12500.npy',trans_list)

# trans_list=[]
# trans_to_kor(back_train['eng_premise'], 'ko',12500,15000)
# np.save(data_path+'premise_eng_to_kor_train_12500_15000.npy',trans_list)

# trans_list=[]
# trans_to_kor(back_train['eng_premise'], 'ko',15000,17500)
# np.save(data_path+'premise_eng_to_kor_train_15000_17500.npy',trans_list)

# trans_list=[]
# trans_to_kor(back_train['eng_premise'], 'ko',17500,20000)
# np.save(data_path+'premise_eng_to_kor_train_17500_20000.npy',trans_list)

# trans_list=[]
# trans_to_kor(back_train['eng_premise'], 'ko',20000,22500)
# np.save(data_path+'premise_eng_to_kor_train_20000_22500.npy',trans_list)

# trans_list=[]
# trans_to_kor(back_train['eng_premise'], 'ko',22500,len(train_df))
# np.save(data_path+'premise_eng_to_kor_train_22500_all.npy',trans_list)
# back0 = np.load(data_path+'premise_eng_to_kor_train_0_2500.npy')
# back1 = np.load(data_path+'premise_eng_to_kor_train_2500_5000.npy')
# back2 = np.load(data_path+'premise_eng_to_kor_train_5000_7500.npy')
# back3 = np.load(data_path+'premise_eng_to_kor_train_7500_10000.npy')
# back4 = np.load(data_path+'premise_eng_to_kor_train_10000_12500.npy')
# back5 = np.load(data_path+'premise_eng_to_kor_train_12500_15000.npy')
# back6 = np.load(data_path+'premise_eng_to_kor_train_15000_17500.npy')
# back7 = np.load(data_path+'premise_eng_to_kor_train_17500_20000.npy')
# back8 = np.load(data_path+'premise_eng_to_kor_train_20000_22500.npy')
# back9 = np.load(data_path+'premise_eng_to_kor_train_22500_all.npy')

# back_train_fin=[*back0,*back1,*back2,*back3,*back4,*back5,*back6,*back7,*back8,*back9]
# back_train_fin=pd.DataFrame(back_train_fin,columns=['kor_premise'])
# back_train_fin=pd.concat([train_df,back_train_fin],axis=1)
# # 0.2내로 너무 짧은 번역
# for i in back_train_fin['kor_premise'].map(len) :
#   no_back_train = back_train_fin[(back_train_fin['kor_premise'].map(len) < i * 0.2)] 
# # Googletrans
# def translate_google_ko(df, col, lang='ko'):
 
#   translator = Translator()
#   df['eng'] = [translator.translate(i, src='en', dest=lang).text for i in df[col]]   # 한영 변환을 새로운 칼럼 'eng'에 담습니다
   
#   tmp1 = df.drop(['eng'], axis=1)                                                    # 2개의 temporary 데이터프레임을 생성해 원본과 한영번역 데이터프레임을 합칩니다
#   tmp2 = df.drop([col], axis=1)
#   tmp2.rename(columns={'eng':'data2'}, inplace=True)
#   result = pd.concat([tmp1,tmp2], ignore_index=True)
#   result = result.drop_duplicates()
#   return result    
# back_train2 = translate_google_en(no_back_train, 'premise')
# back_train2 = back_train2[['index', 'premise', 'hypothesis', 'label', 'kor_premise','data' ]][len(no_back_train):]
# back_train2 = back_train2.reset_index(drop=True)
# back_train3 = translate_google_ko(back_train2, 'data')
# back_train3 = back_train3[['index', 'premise', 'hypothesis', 'label', 'kor_premise','data2' ]][len(no_back_train):]
# back_train3 = back_train3.reset_index(drop=True)
# for i,j in zip(back_train3['index'], range(len(no_back_train))) :
#   back_train_fin['kor_premise'].iloc[i] = back_train3['data2'].iloc[j]
# back_train_fin.to_csv(data_path + "premise_translate_final.csv", index=False)

3. 토크나이징 및 모델 학습

In [15]:
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids = [], [], []

    for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
        encoding_result = tokenizer.encode_plus(sent1, sent2, max_length = max_seq_len, pad_to_max_length = True)

        input_ids.append(encoding_result['input_ids'])
        attention_masks.append(encoding_result['attention_mask'])
        token_type_ids.append(encoding_result['token_type_ids'])

    input_ids = np.array(input_ids, dtype= int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype= int)

    return (input_ids, attention_masks, token_type_ids)

In [16]:
class NLIClassifier(Model):
    def __init__(self, model_name):
        super(NLIClassifier, self).__init__()
        self.bert = TFAutoModel.from_pretrained(model_name, num_labels=3, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(rate=0.5)
        self.classifier = Dense(3, kernel_initializer=TruncatedNormal(0.02),activation='softmax') #가중치 초기화, 활성화함수

    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        input_ids, attention_mask, token_type_ids=inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output, training=training)
        prediction = self.classifier(pooled_output)

        return prediction

In [17]:
def fold_training(train, test_df, model_name):
    cv = StratifiedKFold(n_splits=5, random_state = 2022, shuffle=True)
    cv_pred = []

    for idx, (train_idx, val_idx) in enumerate(cv.split(train[["premise","hypothesis"]], pd.DataFrame(train['label']))):
        X_train, X_val = train[["premise", "hypothesis"]].iloc[train_idx], train[["premise", "hypothesis"]].iloc[val_idx]
        y_train, y_val = pd.DataFrame(train['label']).iloc[train_idx], pd.DataFrame(train['label']).iloc[val_idx]
        
        train_fold = pd.concat([X_train, y_train], axis=1)
        valid_fold = pd.concat([X_val, y_val], axis =1)

        train_fold['premise'] = train_fold['premise'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]', '')
        train_fold['hypothesis'] = train_fold['hypothesis'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]', '')
        valid_fold['premise'] = valid_fold['premise'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]', '')
        valid_fold['hypothesis'] = valid_fold['hypothesis'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]', '')

        tokenizer= AutoTokenizer.from_pretrained(model_name)
        max_seq_len = 128

        X_train = convert_examples_to_features(train_fold['premise'], train_fold['hypothesis'], max_seq_len=max_seq_len, tokenizer = tokenizer)
        X_valid = convert_examples_to_features(valid_fold['premise'], valid_fold['hypothesis'], max_seq_len = max_seq_len, tokenizer = tokenizer)
        X_test = convert_examples_to_features(test_df['premise'], test_df['hypothesis'], max_seq_len = max_seq_len, tokenizer = tokenizer)

        train_label = train_fold['label'].tolist()
        val_label = valid_fold['label'].tolist()
        test_label = test_df['label'].tolist()
        idx_encode = LabelEncoder()
        idx_encode.fit(train_label)

        y_train = idx_encode.transform(train_label) # 주어진 고유한 정수로 변환
        y_val = idx_encode.transform(val_label) # 고유한 정수로 변환

        label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))   #원래 idx와 transform한 idx 딕셔너리로   # classes  => 동일 idx 생략 및 정리
        idx_label = {value:key for key, value in label_idx.items()}

        model = NLIClassifier(model_name)

        optimizer = AdamWeightDecay(1e-5, weight_decay_rate=1e-4)   # 과적합 방지
        loss = tf.keras.losses.SparseCategoricalCrossentropy()
        metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
        model.compile(optimizer = optimizer, loss = loss, metrics = [metric])
        

        early_stopping = EarlyStopping(
            monitor = 'val_accuracy',
            min_delta = 0.001,
            mode = 'max',
            patience =2,
        )

        model.fit(
            X_train, y_train, epochs = 3, batch_size = 8, validation_data = (X_valid, y_val),
            callbacks = [early_stopping]
            )
        
        pred = model.predict(X_test)
        pred_df = pd.DataFrame(pred)
        cv_pred.append(pred_df)
        print('fold{} training finished!'.format(idx+1))

    
    return cv_pred

In [18]:
def training(train, test_df, model_name):
  train, valid = train_test_split(train, test_size=0.2, shuffle=True, stratify=train['label'], random_state = 2022)

  train['premise'] = train['premise'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]', '')
  train['hypothesis'] = train['hypothesis'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]', '')
  valid['premise'] = valid['premise'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]', '')
  valid['hypothesis'] = valid['hypothesis'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]', '')

  tokenizer = AutoTokenizer.from_pretrained(model_name)
  max_seq_len = 128

  X_train = convert_examples_to_features(train['premise'], train['hypothesis'],  max_seq_len=max_seq_len, tokenizer=tokenizer)
  X_valid = convert_examples_to_features(valid['premise'], valid['hypothesis'], max_seq_len=max_seq_len, tokenizer=tokenizer)
  X_test = convert_examples_to_features(test_df['premise'], test_df['hypothesis'],max_seq_len=max_seq_len, tokenizer=tokenizer)

  train_label = train['label'].tolist()
  val_label = valid['label'].tolist()
  test_label = test_df['label'].tolist()
  idx_encode = LabelEncoder()
  idx_encode.fit(train_label)

  y_train = idx_encode.transform(train_label) # 주어진 고유한 정수로 변환
  y_val = idx_encode.transform(val_label) # 고유한 정수로 변환

  label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
  idx_label = {value: key for key, value in label_idx.items()}

  model = NLIClassifier(model_name)

  optimizer = AdamWeightDecay(1e-5, weight_decay_rate=1e-4)  #과적합 방지
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

  early_stopping = EarlyStopping(
      monitor="val_accuracy", 
      min_delta=0.001,
      mode = 'max',
      patience=2)
    
  model.fit(
      X_train, y_train, epochs=3, batch_size = 8, validation_data=(X_valid, y_val),
      )#callbacks = [early_stopping])
    
  pred = model.predict(X_test)
  pred_df = pd.DataFrame(pred)
  return pred_df

In [19]:
test_df['premise'] = test_df['premise'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]", "")
test_df['hypothesis'] = test_df['hypothesis'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


3-1. MODEL 1 : train 원본 + dev 추가 데이터로 학습 (RoBERTa-Large)

In [20]:
# fold_pred_list = fold_training(train_total, test_df, model_name = 'klue/roberta-large')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  from ipykernel import kernelapp as app
Downloading: 100%
375/375 [00:00<00:00, 11.3kB/s]
Downloading: 100%
243k/243k [00:00<00:00, 819kB/s]
Downloading: 100%
734k/734k [00:00<00:00, 803kB/s]
Downloading: 100%
173/173 [00:00<00:00, 3.82kB/s]
  0%|          | 0/22398 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 22398/22398 [00:09<00:00, 2338.15it/s]
100%|██████████| 5600/5600 [00:01<00:00, 2848.60it/s]
100%|██████████| 1666/1666 [00:00<00:00, 5131.96it/s]
Downloading: 100%
547/547 [00:00<00:00, 16.8kB/s]
Downloading: 100%
1.25G/1.25G [00:38<00:00, 37.3MB/s]
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/3
2800/2800 [==============================] - 1495s 517ms/step - loss: 0.7102 - accuracy: 0.6893 - val_loss: 0.4473 - val_accuracy: 0.8214
Epoch 2/3
2800/2800 [==============================] - 1445s 516ms/step - loss: 0.3668 - accuracy: 0.8651 - val_loss: 0.4061 - val_accuracy: 0.8537
Epoch 3/3
2800/2800 [==============================] - 1443s 515ms/step - loss: 0.2333 - accuracy: 0.9165 - val_loss: 0.4580 - val_accuracy: 0.8541
fold1 training finished!
  0%|          | 0/22398 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
100%|██████████| 22398/22398 [00:06<00:00, 3373.98it/s]
100%|██████████| 5600/5600 [00:02<00:00, 2786.68it/s]
100%|██████████| 1666/1666 [00:00<00:00, 4823.59it/s]
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/3
2800/2800 [==============================] - 1486s 516ms/step - loss: 0.6186 - accuracy: 0.7440 - val_loss: 0.4114 - val_accuracy: 0.8339
Epoch 2/3
2800/2800 [==============================] - 1441s 515ms/step - loss: 0.3341 - accuracy: 0.8783 - val_loss: 0.3971 - val_accuracy: 0.8525
Epoch 3/3
2800/2800 [==============================] - 1441s 515ms/step - loss: 0.2050 - accuracy: 0.9263 - val_loss: 0.5276 - val_accuracy: 0.8516
fold2 training finished!
  0%|          | 0/22398 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
100%|██████████| 22398/22398 [00:06<00:00, 3296.55it/s]
100%|██████████| 5600/5600 [00:01<00:00, 3280.86it/s]
100%|██████████| 1666/1666 [00:00<00:00, 4765.44it/s]
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/3
2800/2800 [==============================] - 1487s 516ms/step - loss: 0.6405 - accuracy: 0.7366 - val_loss: 0.4754 - val_accuracy: 0.8154
Epoch 2/3
2800/2800 [==============================] - 1442s 515ms/step - loss: 0.3493 - accuracy: 0.8721 - val_loss: 0.3912 - val_accuracy: 0.8507
Epoch 3/3
2800/2800 [==============================] - 1442s 515ms/step - loss: 0.2149 - accuracy: 0.9250 - val_loss: 0.4929 - val_accuracy: 0.8446
fold3 training finished!
  0%|          | 0/22399 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
100%|██████████| 22399/22399 [00:06<00:00, 3400.67it/s]
100%|██████████| 5599/5599 [00:01<00:00, 3394.85it/s]
100%|██████████| 1666/1666 [00:00<00:00, 4748.73it/s]
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/3
2800/2800 [==============================] - 1516s 527ms/step - loss: 1.1502 - accuracy: 0.3338 - val_loss: 1.1047 - val_accuracy: 0.3415
Epoch 2/3
2800/2800 [==============================] - 1467s 524ms/step - loss: 1.1288 - accuracy: 0.3368 - val_loss: 1.1084 - val_accuracy: 0.3195
Epoch 3/3
2800/2800 [==============================] - 1468s 524ms/step - loss: 1.1239 - accuracy: 0.3364 - val_loss: 1.1040 - val_accuracy: 0.3390
fold4 training finished!
  0%|          | 0/22399 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
100%|██████████| 22399/22399 [00:06<00:00, 3429.12it/s]
100%|██████████| 5599/5599 [00:01<00:00, 3441.68it/s]
100%|██████████| 1666/1666 [00:00<00:00, 5026.51it/s]
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/3
2800/2800 [==============================] - 1521s 528ms/step - loss: 0.9690 - accuracy: 0.5159 - val_loss: 0.6331 - val_accuracy: 0.7516
Epoch 2/3
2800/2800 [==============================] - 1471s 525ms/step - loss: 0.5095 - accuracy: 0.8063 - val_loss: 0.4842 - val_accuracy: 0.8198
Epoch 3/3
2800/2800 [==============================] - 1471s 525ms/step - loss: 0.3144 - accuracy: 0.8879 - val_loss: 0.4804 - val_accuracy: 0.8323
fold5 training finished!


3-4. MODEL 4 : 해석 데이터를 추가하지 않은 원본 데이터로 학습 (Pytorch, KoELECTRA)

In [21]:
MODEL_NAME = 'monologg/koelectra-base-v3-discriminator'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
config = AutoConfig.from_pretrained(MODEL_NAME)
config.num_labels = 3

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config)

train_dataset, eval_dataset = train_test_split(train_total, test_size=0.2, shuffle=True, stratify=train_total['label'], random_state = 2022)

tokenized_train = tokenizer(
    list(train_dataset['premise']),
    list(train_dataset['hypothesis']),
    return_tensors="pt",
    max_length=128, 
    padding=True,
    truncation=True,
    add_special_tokens=True
)

tokenized_eval = tokenizer(
    list(eval_dataset['premise']),
    list(eval_dataset['hypothesis']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

tokenized_test = tokenizer(
    list(test_df['premise']),
    list(test_df['hypothesis']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/431M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [22]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['label'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [23]:
def label_to_num(label):
    label_dict = {"contradiction": 0, "entailment": 1, "neutral": 2, "answer": 3}
    num_label = []

    for v in label:
        num_label.append(label_dict[v])
    
    return num_label
train_label = label_to_num(train_dataset['label'].values)
eval_label = label_to_num(eval_dataset['label'].values)
test_label = label_to_num(test_df['label'].values)

train_dataset = BERTDataset(tokenized_train, train_label)
eval_dataset = BERTDataset(tokenized_eval, eval_label)
test_dataset = BERTDataset(tokenized_test, test_label)

In [24]:
def compute_metrics(pred):
  """ validation을 위한 metrics function """
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  probs = pred.predictions

  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds) # 리더보드 평가에는 포함되지 않습니다.

  return {
      'accuracy': acc,
  }

In [25]:
# 모델 초모수 수정하기
training_ars = TrainingArguments(
    output_dir='./result',
    num_train_epochs=7,
    per_device_train_batch_size=32, # 64
    learning_rate=5e-5,
    weight_decay=0.1,
    save_total_limit=5,
    save_steps=500,
    evaluation_strategy='steps',
    eval_steps = 500,
    load_best_model_at_end = True,
)

trainer = Trainer(
    model=model,
    args=training_ars,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 22398
  Num Epochs = 7
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4900


Step,Training Loss,Validation Loss,Accuracy
500,0.531400,0.378810,0.861250
1000,0.305200,0.411007,0.856964
1500,0.205000,0.547196,0.864107
2000,0.123800,0.550713,0.867143
2500,0.083900,0.692264,0.871429
3000,0.058200,0.701603,0.876786
3500,0.042500,0.712986,0.881071
4000,0.019600,0.784401,0.882143
4500,0.017500,0.811361,0.881429


***** Running Evaluation *****
  Num examples = 5600
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-500
Configuration saved in ./result/checkpoint-500/config.json
Model weights saved in ./result/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5600
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-1000
Configuration saved in ./result/checkpoint-1000/config.json
Model weights saved in ./result/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5600
  Batch size = 8
Saving model checkpoint to ./result/checkpoint-1500
Configuration saved in ./result/checkpoint-1500/config.json
Model weights s

TrainOutput(global_step=4900, training_loss=0.1423507296309179, metrics={'train_runtime': 1783.6227, 'train_samples_per_second': 87.903, 'train_steps_per_second': 2.747, 'total_flos': 7895986387825032.0, 'train_loss': 0.1423507296309179, 'epoch': 7.0})

In [26]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Tokenizer_NAME = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(Tokenizer_NAME)

MODEL_NAME = './result/checkpoint-4500'
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(tokenizer.vocab_size)
model.to(device)

loading configuration file https://huggingface.co/monologg/koelectra-base-v3-discriminator/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/353ae9c3d9daa722551a585351b95934cefcf83155f6ff52a1975fa27863dfe0.9c57bd1e7b894b078a3a8ed91a498ca5fb48334c137fe2ec43e8079db1878f8c
Model config ElectraConfig {
  "_name_or_path": "monologg/koelectra-base-v3-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "t

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [27]:
dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model.eval()
output_pred = []
output_prob = []

for i, data in enumerate(tqdm(dataloader)):
    with torch.no_grad():
        outputs = model(
            input_ids=data['input_ids'].to(device),
            attention_mask=data['attention_mask'].to(device),
            token_type_ids=data['token_type_ids'].to(device)
        )
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result)
    output_prob.append(prob)
  
pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
pred_df2 = pd.DataFrame(output_prob)

100%|██████████| 105/105 [00:04<00:00, 22.62it/s]


In [28]:
# train1_fold1 = fold_pred_list[0]
# train1_fold2 = fold_pred_list[1]
# train1_fold3 = fold_pred_list[2]
# pred_ensemble = (train1_fold1 + train1_fold2 + train1_fold3)/3
# label_idx = {'contradiction':0, 'entailment':1, 'neutral':2}
# result = [np.argmax(val) for val in np.array(pred_ensemble)]
# out = [list(label_idx.keys())[_] for _ in result]
# zip_dir = '/content/drive/MyDrive/화요일 스터디 폴더/dacon'
# sub_path = zip_dir + "/data/sample_submission.csv"
# submission = pd.read_csv(sub_path)
# submission['label']=out
# submission.to_csv(zip_dir + "/data/ensemble_final.csv", index=False)


In [30]:
# train1_fold1.to_csv(zip_dir + "/data/fold1.csv", index=False)
# train1_fold2.to_csv(zip_dir + "/data/fold2.csv", index=False)
# train1_fold3.to_csv(zip_dir + "/data/fold3.csv", index=False)
# train1_fold4.to_csv(zip_dir + "/data/fold4.csv", index=False)
# train1_fold5.to_csv(zip_dir + "/data/fold5.csv", index=False)
pred_df2.to_csv(zip_dir + "/data/pred_df2.csv", index=False)

In [31]:
train1_fold1 = pd.read_csv(zip_dir + "/data/fold1.csv")
train1_fold2 = pd.read_csv(zip_dir + "/data/fold2.csv")
train1_fold3 = pd.read_csv(zip_dir + "/data/fold3.csv")
pred_df2 = pd.read_csv(zip_dir + "/data/pred_df2.csv")

In [32]:
train4 = pred_df2
pred_ensemble = (train1_fold1 + train1_fold2 + train1_fold3 + train4)/4
label_idx = {'contradiction':0, 'entailment':1, 'neutral':2}
result = [np.argmax(val) for val in np.array(pred_ensemble)]
out = [list(label_idx.keys())[_] for _ in result]
zip_dir = '/content/drive/MyDrive/화요일 스터디 폴더/dacon'
sub_path = zip_dir + "/data/sample_submission.csv"
submission = pd.read_csv(sub_path)
submission['label']=out
submission.to_csv(zip_dir + "/data/ensemble_final2.csv", index=False)
